<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/code/sergioadauto/da-covid-brazil/edit) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/u/0/reporting/4bbdec07-ad4f-4347-824d-3fd0741f97d2/page/J9vAD).

---

# **COVID Dashboard**

## 1\. Contexto


<p style="text-align:justify"> Conforme o <a href="https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus">Ministério da Saúde</a>, a Covid-19 é uma doença infecto respiratória causada pelo vírus da SARS-CoV-2, que apresenta elevado potencial de transmissibilidade de impacto global. Que afetou o mundo todo, e gerou mudanças de comportamento a respeito da precaução/proteção de doenças infectocontagiosas.</p>
<p style="text-align:justify">Por esse motivo, o estudo dos dados de transmissão e evolução dos casos em uma região, é fundamental para garantir o combate e controle da doença. Essa projeto visa construir um dashboard de visualização interativa, a respeito dos dados de vacinação e casos de Covid-19 no Brasil. Para esses dados serem consumidos, será utilizado o método de ETL (Extração - Transformação - Carregamento), e depois visualizados na plataforma Google Data Studio.</p>
<p style="text-align:justify">Para fazer isso, serão utilizadas duas bases de dados, a primeira vai conter informações sobre os casos da COVID-19, e a segunda informações de registro das vacinações. A primeira base (Casos de Covid-19), trata-se de um compilado de informações coletadas diariamente desde janeiro de 2020 pelo centro de ciência e engenharia da universidade <a href="https://www.jhu.edu/">John Hopkins</a>, que apresenta descrições geográficas de regiões de países (como estados, condados, províncias, etc.), além disso, seus dados podem ser consultados em seu <a href="https://systems.jhu.edu/research/public-health/ncov/">portal</a>, ou <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports">repositório</a> público. A segunda base de dados (Registro de Vacinação), refere-se a um compilado desenvolvido pelo projeto Nosso Mundo em Dados da universidade de <a href="https://www.ox.ac.uk/">Oxford</a>, que iniciou em janeiro de 2020, sendo atualizado diariamente com características geográficas de países, os dados podem ser acessados pelo <a href="https://ourworldindata.org">portal - Our World in Data</a>, ou através desse <a href="https://covid.ourworldindata.org/data/owid-covid-data.csv">link</a>.</p>

<h3>Descrição dos dados:</h3>
<p><strong>Caso de COVID-19:</strong></p>
    <ul>
         <li><em>date</em> = data de referência;
         <li><em>state</em> = estado;
         <li><em>country</em>: país; 
         <li><em>population</em> = população estimada;
         <li><em>confirmed</em> = número acumulado de infectados;
         <li><em>confirmed_1d</em> = número diário de infectados;
         <li><em>confirmed_moving_avg_7d</em> = média móvel de 7 dias do número diário de infectados;
         <li><em>confirmed_moving_avg_7d_rate_14d</em> = média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
         <li><em>deaths</em> = número acumulado de mortos;
         <li><em>deaths_1d</em> = número diário de mortos;
         <li><em>deaths_moving_avg_7d</em> = média móvel de 7 dias do número diário de mortos;
         <li><em>deaths_moving_avg_7d</em> = média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
         <li><em>month</em> = mês de referência;
         <li><em>year</em> = ano de referência.
    </ul>
<p><strong>Registros de Vacinação da COVID-19:</strong></p>
    <ul>
        <li><em>date</em> = data de referência;
        <li><em>country</em> = país;
        <li><em>population</em> = população estimada;
        <li><em>total</em> = número acumulado de doses administradas;
        <li><em>one_shot</em> = número acumulado de pessoas com uma dose;
        <li><em>one_shot_perc</em> = número acumulado relativo de pessoas com uma dose;
        <li><em>two_shots</em> = número acumulado de pessoas com duas doses;
        <li><em>two_shot_perc</em> = número acumulado relativo de pessoas com duas doses;
        <li><em>three_shots</em> = número acumulado de pessoas com três doses;
        <li><em>three_shot_perc</em> = número acumulado relativo de pessoas com três doses;
        <li><em>month</em> = mês de referência;
        <li><em>year</em> = ano de referência.
    </ul>




## 2\. Pacotes e bibliotecas

In [ ]:
# importe todas as suas bibliotecas aqui, siga os padrões do PEP8:
#
# - 1º pacotes nativos do python: json, os, etc.;
# - 2º pacotes de terceiros: pandas, seabornm etc.;
# - 3º pacotes que você desenvolveu.
#

from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

<p>Antes de começar extrair os dados, será preciso definir um intervalo de tempo para trabalhar com as duas bases de dados (<em>Casos de Covid-19</em> e <em>Registro de Vacinação</em>). Minha ideia é utilizar as informações referente aos anos de 2021 e 2022, mas para isso, preciso identificar um intervalo máximo de tempo, que seja equivalente para trabalhar nas duas bases.</p>
<p>Com base nas informações presente no <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports">repositório</a> de casos de covid-19, a informação de registro mais recente disponível no momento é de <code>05-12-2022</code>. Diante dessa informação, vou utilizar-lá como parâmetro máximo para intervalo de tempo a ser trabalhado com as duas bases de dados.</p>
<details>
    <summary>Click aqui para visualizar o print do repositório</summary>
<img src="https://github.com/SergioAdauto/da-covid-brazil/blob/main/image/print.png?raw=true" width="80%">
</details>

In [ ]:
# faça o código de extração dos dados:
#
# - coleta de dados;
# - wrangling da estrutura;
# - exploração do schema;
# - etc.

# Função responsável por definir um intervalo de tempo:
def intervalo_tempo(data_inicio: datetime, data_fim: datetime) -> Iterator[datetime]:
  intervalo_tempo_dias: int = (data_fim - data_inicio).days
  for data in range(intervalo_tempo_dias):
    yield data_inicio + timedelta(data)


# Definindo o intervalo de tempo:
data_inicio = datetime(2021,  1,  1)
data_fim = datetime(2022,  12,  5)

#Variável para registrar o Dataframe de casos de covid-19:
df_casos = None

casos_vazio = True # Variável responsável por validar se o dataframe está concluido.

#Loop para extrair os dados dados do repositório e selecionar as colunas desejadas:
for data in intervalo_tempo(data_inicio, data_fim):
    # Variável responsável por consultar a data desejada no repositório:  
    data_consulta = data.strftime('%m-%d-%Y')
    # link do repositório:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{data_consulta}.csv'
    # Carregando os dados de casos do dia em um dataframe:
    caso = pd.read_csv(filepath_or_buffer=url, sep=',')
    # Apagando as colunas que não serão utilizadas:
    caso = caso.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    # Selecionando as informações do Brasil:
    caso = caso.query('Country_Region == "Brazil"').reset_index(drop=True)
    # Convertendo a coluna 'Date' para o formato de data:
    caso['Date'] = pd.to_datetime(data.strftime('%Y-%m-%d'))
    #Verificando se o dataframe está completo: 
    if casos_vazio:
        df_casos = caso
        casos_vazio = False
    else:
        df_casos = df_casos.append(caso, ignore_index=True)

#Carregando o DataFrame do Registro de vacinação:
df_vacinacao = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)



## 4\. Transformação

In [ ]:
# faça o código de manipulação dos dados:
#
# - enriquecimento;
# - controle de qualidade;
# - etc.

# Renomeando as colunas "Province_State" e "Country_Region" para
# facilitar sua compreensão:
df_casos = df_casos.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

# Deixando colunas em minúscula:
for column in df_casos.columns:
  df_casos.rename(columns={column: column.lower()}, inplace=True)

# Verificando o nome dos estados:
df_casos.state.unique()


# OBS: Como alguns estados brasileiros estão com seu nome sem acentuação, será 
# preciso ajustar isso para facilitar a geolocalização do mesmo.
# Dicionário com nome dos estados que possuem acentuação:
estados = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}
# Ajustando nome de alguns estados brasileiros:
df_casos['state'] = df_casos['state'].apply(lambda state: estados.get(state) if state in estados.keys() else state)

# Criando coluna de Ano:
df_casos['year']  = df_casos['date'].apply(lambda date: date.strftime('%Y'))

# Criando coluna de Mês:
df_casos['month'] = df_casos['date'].apply(lambda date: date.strftime('%Y-%m'))

#Criando coluna da população estimada 
df_casos['population'] = round(100000 * (df_casos['confirmed'] / df_casos['incident_rate']))

#Eliminando a coluna de taxa de incidentes:
df_casos.drop('incident_rate', axis=1, inplace=True)



#Variável para registrar os casos:
casos = None
# Para testar os casos vazio:
casos_vazio = True

#Função para verificar a tendência dos casos:
def tendencia(taxa: float) -> str:
    if np.isnan(taxa):
        return np.NaN

    if taxa < 0.75:
        status = 'redução'
    elif taxa > 1.15:
        status = 'aumento'
    else:
        status = 'estável'

    return status
  
# Loop para verificar os casos por estados brasileiro:
for state in df_casos['state'].drop_duplicates():
    casos_por_estado = df_casos.query(f'state == "{state}"').reset_index(drop=True)
    casos_por_estado = casos_por_estado.sort_values(by=['date'])
    # Casos confirmados:
    casos_por_estado['confirmed_1d'] = casos_por_estado['confirmed'].diff(periods=1)
    casos_por_estado['confirmed_moving_avg_7d'] = np.ceil(casos_por_estado['confirmed_1d'].rolling(window=7).mean())
    casos_por_estado['confirmed_moving_avg_7d_rate_14d'] = casos_por_estado['confirmed_moving_avg_7d']/casos_por_estado['confirmed_moving_avg_7d'].shift(periods=14)
    casos_por_estado['confirmed_trend'] = casos_por_estado['confirmed_moving_avg_7d_rate_14d'].apply(tendencia)
    # Casos que registraram mortes:
    casos_por_estado['deaths_1d'] = casos_por_estado['deaths'].diff(periods=1)
    casos_por_estado['deaths_moving_avg_7d'] = np.ceil(casos_por_estado['deaths_1d'].rolling(window=7).mean())
    casos_por_estado['deaths_moving_avg_7d_rate_14d'] = casos_por_estado['deaths_moving_avg_7d']/casos_por_estado['deaths_moving_avg_7d'].shift(periods=14)
    casos_por_estado['deaths_trend'] = casos_por_estado['deaths_moving_avg_7d_rate_14d'].apply(tendencia)
    
    # Verificando se os casos por estado estão registrado como vazio:
    if casos_vazio:
        casos = casos_por_estado
        casos_vazio = False
    else:
        casos = casos.append(casos_por_estado, ignore_index=True)
# Salva as informações no DataFrame:
df_casos = casos
casos = None


# Ajustando os tipos das colunas criadas:
df_casos = df_casos.astype({'population':np.int64, 'confirmed_1d':np.int64, 'confirmed_moving_avg_7d':np.int64, 'deaths_1d':np.int64,'deaths_moving_avg_7d':np.int64}, errors='ignore')

# Reorganizando as colunas:
df_casos = df_casos[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]




# Transformação dos dados de vacinação:
# Selecionando os elementos do Brasil:
df_vacinacao = df_vacinacao.query('location == "Brazil" and date >= "2021-01-01" and date <= "2022-12-04"').reset_index(drop=True)

# Selecionado as colunas do dataframe de vacinação:
df_vacinacao = df_vacinacao[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

# Verificando se existe elementos faltando:
df_vacinacao.isna().sum()

# Replicando os elementos faltantes com as informações da linha anterior:
df_vacinacao = df_vacinacao.fillna(method='ffill')

# Reorganizando o DataFrame de Vacinação:
df_vacinacao.rename(columns={'location': 'country',
                                            'total_vaccinations': 
                                            'total','people_vaccinated': 'one_shot',
                                            'people_fully_vaccinated': 'two_shots',
                                            'total_boosters': 'three_shots'}, inplace=True)


# Criando uma coluna ano:
df_vacinacao['year']  = df_vacinacao['date'].apply(lambda date: date.strftime('%Y'))

#Criando coluna de mês:
df_vacinacao['month'] = df_vacinacao['date'].apply(lambda date: date.strftime('%Y-%m'))

#Montando as colunas de proporções de dose de vacinação (1ª, 2ª e 3ª dose):
df_vacinacao['one_shot_perc'] = round(df_vacinacao['one_shot'] / df_vacinacao['population'], 4)
df_vacinacao['two_shots_perc'] = round(df_vacinacao['two_shots'] / df_vacinacao['population'], 4)
df_vacinacao['three_shots_perc'] = round(df_vacinacao['three_shots'] / df_vacinacao['population'], 4)

#verificando o tipos dos dados:
df_vacinacao.info()

# Ajustando os tipos das colunas:
df_vacinacao.astype({'population':np.int64, 'total':np.int64, 'one_shot':np.int64, 'two_shots':np.int64, 'three_shots':np.int64}, errors='ignore')

#Organizando o DataFrame:
df_vacinacao = df_vacinacao[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]


## 5\. Carregamento

In [ ]:
# faça o código para persistência dos dados:
#
# - salve-os no formato csv sem índice;
# - etc.


# Após manipular todos os dados de casos de covid e vacinação, agora será persistido em disco os dois DataFrames, para assim serem utilizados no Google Data Studio.
df_casos.to_csv('./covid-cases.csv', sep=',', index=False)
df_vacinacao.to_csv('./covid-vaccines.csv', sep=',', index=False)